In [1]:
import random

#Pennylane
import pennylane as qml
from pennylane import numpy as np

#PyTorch
import torch
import torchvision
from torchvision import datasets, transforms

In [2]:
dev = qml.device('default.qubit', wires = 6)
n_qubits = 6
n_layer_steps = 7
n_layers_to_add = 2
dim = 2**(n_qubits)

In [3]:
def set_random_gates(n_qubits: int):
    
    gate_set = [qml.RX, qml.RY, qml.RZ]
    chosen_gates = []
    for i in range(n_qubits):
        chosen_gate = random.choice(gate_set)
        chosen_gates.append(chosen_gate)
    return chosen_gates

def norm(complex_vector: np.array) -> float:
    """Returns the norm of a complex vector.
    
    Args:
       complex_vector (np.arrray of shape (dim,)):
           complex vector with an arbitrary dimension.
           
       
    Returns:
        norm (float): norm or magnitude of the complex vector.
            
    """
    
    #define the norm of a complex vector here
    norm = np.sqrt(sum([np.square(np.absolute(complex_vector[i])) for i in range(complex_vector.shape[0])]))
    
    return norm

def random_quantum_state(dim: int, amplitude_type: str = 'complex') -> np.array:
    """Creates a normalized random real or complex vector of a defined
    dimension.
    
    Args:
        dim (int): integer number specifying the dimension
            of the vector that we want to generate.
            
        amplitude_type (str): string indicating if you want to create a
            vector with 'complex' or 'real' number amplitudes.
            
    Returns:
        (np.array): normalized real or complex vector of the given
            dimension.
    """
    #generate an unnormalized complex vector of dimension = dim
    if amplitude_type == 'complex':
        Z = np.random.random(dim) + np.random.random(dim)*1j
    
    #generate an unnormalized real vector of dimension = dim
    elif amplitude_type == 'real':
        Z = np.random.random(dim)
    
    #normalize the complex vector Z
    Z /= norm(Z)
    
    return Z

def density_matrix(target_vector: np.array) -> np.array:
    """Return the density matrix of a pure state.
    
    Arguments:
        target_vector (np.array): Complex vector array.
        
    Returns:
        (np.array): Density matrix of shape (dim, dim),
            where dim is the dimension of the target_vector.
    """
    target_vector_reshape = target_vector.reshape((target_vector.shape[0],1))
    density_matrix = target_vector_reshape @ np.transpose(np.conjugate(target_vector_reshape))
    
    return density_matrix

In [4]:
target_vector = random_quantum_state(dim)
density_matrix = density_matrix(target_vector)

### Templates for Phase I

In [5]:
@qml.template
def apply_layer(gates, weights):
    
    for i in range(n_qubits):
        gates[i](weights[i], wires = i)
    
    tuples = [(i,i+1) for i in range(n_qubits-1)]

    for tup in tuples:
        qml.CZ(wires=[tup[0], tup[1]])
        
@qml.template #template for non-trainable part of the quantum circuit
def frozen_layers(frozen_layer_gates, frozen_layer_weights): 
    for i in range(len(frozen_layer_gates)):
        apply_layer(frozen_layer_gates[i], frozen_layer_weights[i])

@qml.qnode(dev)
def trainable_circuit(params):
    
    #apply frozen layers
    frozen_layers(layer_gates, layer_weights)
    
    #apply trainable layers
    new_weights = []
    for i in range(n_layers_to_add):
        new_weights.append(params[int(i*n_qubits):int((i+1)*n_qubits)])

    for i in range(n_layers_to_add):
        apply_layer(new_gates[i], new_weights[i])
        
    wirelist = [i for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix, wirelist))

In [6]:
def cost(x):
    
    return (1.0 - trainable_circuit(x))

In [7]:
layer_gates = []
layer_weights = []
n_layers = 2
for i in range(n_layers):
    layer_gates.append(set_random_gates(n_qubits))
    layer_weights.append(np.random.rand(n_qubits)*2*np.pi)
    
new_gates = [set_random_gates(n_qubits) for i in range(n_layers_to_add)]
new_weights = np.random.rand(n_qubits*n_layers_to_add)*2*np.pi

trainable_circuit(new_weights)

0.00013232665103589305

In [8]:
print(trainable_circuit.draw())

 0: ──RX(5.24)────╭C───RZ(0.239)─────────────╭C──────────RZ(2.489)──────────────╭C───────────RY(3.732)──────────────╭C─────────────────────────────────────────╭┤ ⟨H0⟩ 
 1: ──RY(4.79)────╰Z──╭C───────────RX(2.91)──╰Z─────────╭C───────────RY(0.717)──╰Z──────────╭C───────────RY(4.301)──╰Z──────────╭C─────────────────────────────├┤ ⟨H0⟩ 
 2: ──RX(3.31)────────╰Z──────────╭C──────────RZ(1.19)──╰Z──────────╭C───────────RX(6.155)──╰Z──────────╭C───────────RX(4.094)──╰Z─────────╭C──────────────────├┤ ⟨H0⟩ 
 3: ──RZ(4.49)────────────────────╰Z─────────╭C──────────RY(3.06)───╰Z──────────╭C───────────RZ(2.405)──╰Z──────────╭C───────────RY(4.97)──╰Z──────────╭C──────├┤ ⟨H0⟩ 
 4: ──RZ(5.99)───────────────────────────────╰Z─────────╭C───────────RY(0.901)──╰Z──────────╭C───────────RX(0.473)──╰Z──────────╭C──────────RX(3.822)──╰Z──╭C──├┤ ⟨H0⟩ 
 5: ──RZ(0.0878)────────────────────────────────────────╰Z───────────RZ(4.81)───────────────╰Z───────────RY(1.135)──────────────╰Z──────────RX(0.277)──────╰Z──╰

In [9]:
layer_gates

[[pennylane.ops.qubit.RX,
  pennylane.ops.qubit.RY,
  pennylane.ops.qubit.RX,
  pennylane.ops.qubit.RZ,
  pennylane.ops.qubit.RZ,
  pennylane.ops.qubit.RZ],
 [pennylane.ops.qubit.RZ,
  pennylane.ops.qubit.RX,
  pennylane.ops.qubit.RZ,
  pennylane.ops.qubit.RY,
  pennylane.ops.qubit.RY,
  pennylane.ops.qubit.RZ]]

In [ ]:
layer_gates = []
layer_weights = []
cost_list = []
for step in range(n_layer_steps):
    
    new_gates = [set_random_gates(n_qubits) for i in range(n_layers_to_add)]
    init_params = np.zeros(n_qubits*n_layers_to_add)
    
    opt = qml.GradientDescentOptimizer(stepsize = 0.4)

    opt_steps = 100

    params = init_params
    
    for i in range(opt_steps):

        params = opt.step(cost, params)
    
    
    print(step)
    print(layer_weights)
    print(cost(params))
    cost_list.append(cost(params))
    
    trained_weights = [params[int(i*n_qubits):int((i+1)*n_qubits)] for i in range(n_layers_to_add)]
    
    layer_gates += new_gates
    layer_weights += trained_weights

0
[]
0.9024292206536404
1
[array([ 1.65143805e-01, -7.93335605e-02,  6.93889390e-19,  1.50238624e-01,
        1.47769586e-01, -3.30400957e-01]), array([ 0.14454178, -0.04168883,  1.15194764, -0.03582504,  1.01763943,
       -0.13664354])]
0.7829554378549575
2
[array([ 1.65143805e-01, -7.93335605e-02,  6.93889390e-19,  1.50238624e-01,
        1.47769586e-01, -3.30400957e-01]), array([ 0.14454178, -0.04168883,  1.15194764, -0.03582504,  1.01763943,
       -0.13664354]), array([ 0.83976508, -0.76299429, -0.3773989 ,  1.10658517,  0.35429636,
       -0.25847681]), array([ 0.49518756,  0.14207511, -0.34694144,  0.19335861,  0.21768019,
       -0.25847681])]
0.5075206167270266
3
[array([ 1.65143805e-01, -7.93335605e-02,  6.93889390e-19,  1.50238624e-01,
        1.47769586e-01, -3.30400957e-01]), array([ 0.14454178, -0.04168883,  1.15194764, -0.03582504,  1.01763943,
       -0.13664354]), array([ 0.83976508, -0.76299429, -0.3773989 ,  1.10658517,  0.35429636,
       -0.25847681]), array([ 0.4

In [ ]:
layer_weights